To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [24]:
%%writefile dataset.csv
instruction,input,output
"Extract from the query.","I want a 2BHK flat for 50 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","Looking for a 3BHK apartment with a price of 75 lakhs","SELECT * FROM properties WHERE bedroom_count = 3 AND price <= 7500000;"
"Extract from the query.","Need a 4BHK villa under 1 crore","SELECT * FROM properties WHERE bedroom_count = 4 AND price <= 10000000;"
"Extract from the query.","I'm searching for a 1BHK condo for around 30k","SELECT * FROM properties WHERE bedroom_count = 1 AND price <= 30000;"
"Extract from the query.","Interested in a 3BHK flat with a budget of 80 lakhs","SELECT * FROM properties WHERE bedroom_count = 3 AND price <= 8000000;"
"Extract from the query.","I need a 2BHK apartment, priced at 60 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 6000000;"
"Extract from the query.","Looking for a luxurious 4BHK apartment at 1.5 crores","SELECT * FROM properties WHERE bedroom_count = 4 AND price <= 15000000;"
"Extract from the query.","I want a compact 1BHK flat costing 25 lakhs","SELECT * FROM properties WHERE bedroom_count = 1 AND price <= 2500000;"
"Extract from the query.","Find me a 3BHK property with a price tag of 90 lakhs","SELECT * FROM properties WHERE bedroom_count = 3 AND price <= 9000000;"
"Extract from the query.","I'm interested in a 2BHK apartment priced at 55 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5500000;"
"Extract from the query.","Looking for a 2BHK unit for approximately 65 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 6500000;"
"Extract from the query.","Can you get me a three-bedroom flat costing around 85 lakhs?","SELECT * FROM properties WHERE bedroom_count = 3 AND price <= 8500000;"
"Extract from the query.","I need a single-bedroom condo for about 40 lakhs","SELECT * FROM properties WHERE bedroom_count = 1 AND price <= 4000000;"
"Extract from the query.","Find a duplex property available at a price of 1.2 crores","SELECT * FROM properties WHERE description ILIKE '%duplex%' AND price <= 12000000;"
"Extract from the query.","Seeking a 4BHK apartment with a cost of 1 crore rupees","SELECT * FROM properties WHERE bedroom_count = 4 AND price <= 10000000;"
"Extract from the query.","I'm looking for a studio apartment priced at 20 lakhs","SELECT * FROM properties WHERE description ILIKE '%studio%' AND price <= 2000000;"
"Extract from the query.","Need a penthouse for a 3BHK at around 2 crores","SELECT * FROM properties WHERE description ILIKE '%penthouse%' AND bedroom_count = 3 AND price <= 20000000;"
"Extract from the query.","I want a 2BHK flat for about 70L","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 7000000;"
"Extract from the query.","Searching for a 3BHK home with a budget of nearly 95 lakhs","SELECT * FROM properties WHERE bedroom_count = 3 AND price <= 9500000;"
"Extract from the query.","Looking for a 1BHK apartment that costs roughly 35,000 rupees","SELECT * FROM properties WHERE bedroom_count = 1 AND price <= 35000;"
"Extract from the query.","I am in need of a 2BHK apartment; my budget is 55,000,000 INR","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 55000000;"
"Extract from the query.","Looking for a 3BHK apartment with an approximate price of seventy-five lakhs","SELECT * FROM properties WHERE bedroom_count = 3 AND price <= 7500000;"
"Extract from the query.","Seeking a 4BHK house available for 1.25 crores","SELECT * FROM properties WHERE bedroom_count = 4 AND price <= 12500000;"
"Extract from the query.","I require a 1BHK flat, the cost should be around 28k","SELECT * FROM properties WHERE bedroom_count = 1 AND price <= 28000;"
"Extract from the query.","Looking for a 2BHK residence with a price near 60 lakhs rupees","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 6000000;"
"Extract from the query.","I want a 2rooms house for 50 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","Looking for a property with 2 rooms and a hall, priced at 60 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 6000000;"
"Extract from the query.","Need a flat with 2rooms with bathroom and hall, price around 55 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5500000;"
"Extract from the query.","I want a 3 and half BHK apartment for 80 lakhs","SELECT * FROM properties WHERE bedroom_count = 3.5 AND price <= 8000000;"
"Extract from the query.","Looking for a 3.5 BHK house costing 90 lakhs","SELECT * FROM properties WHERE bedroom_count = 3.5 AND price <= 9000000;"
"Extract from the query.","Need a property with 3 and a half rooms for around 85 lakhs","SELECT * FROM properties WHERE bedroom_count = 3.5 AND price <= 8500000;"
"Extract from the query.","I need a two BHK apartment for 50 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","Looking for a 2 bhk property for ₹50 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","Need a 2BHK apartment, price is negotiable","SELECT * FROM properties WHERE bedroom_count = 2 ;"
"Extract from the query.","I need a 2BHK apartment between 50 and 60 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price BETWEEN 5000000 AND 6000000;"
"Extract from the query.","Looking for a 2BHK priced at 50-60 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price BETWEEN 5000000 AND 6000000;"
"Extract from the query.","I want a 2BHK property for 50,00,000 rupees","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","Need a 2BHK with extra storeroom for 50L","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","I need a two-bedroom, hall, and kitchen apartment for about 50 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","Looking for a 2 BHK, recently renovated, priced at 55 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5500000;"
"Extract from the query.","Need a 2BHK apartment with price as per market rates","SELECT * FROM properties WHERE bedroom_count = 2 ;"
"Extract from the query.","I need a 2BHK apartment for 50 lakhs","SELECT * FROM properties WHERE bedroom_count = 2 AND price <= 5000000;"
"Extract from the query.","I am looking for an occupancy-ready resale unit by Brigade in 'Brigade Valley'. It's 3 years old, partially furnished, around 75 lakhs, described as modern and spacious, facing east, with 1 dedicated parking spot, visitor limit of 4 per slot, built-up area of 1300 sqft, lake view, contemporary furnishing style, building named 'Brigade Crest', and must have a gym and park.","SELECT * FROM properties WHERE ready_to_movin = FALSE AND developer_name = 'Brigade' AND project_name = 'Brigade Valley' AND resale = TRUE AND age = 3 AND furnished ILIKE '%partially furnished%' AND price = 7500000 AND tags ILIKE '%modern%' AND tags ILIKE '%spacious%' AND facing = 'east' AND parkingCount = 1 AND visitor_perslot = 4 AND buildup_area = 1300 AND property_view = 'lake view' AND property_furnished_type ILIKE '%contemporary%' AND buildingName = 'Brigade Crest' AND location ILIKE '%Brigade Valley%' AND must_have_tags ILIKE '%gym%' AND must_have_tags ILIKE '%park%';"
"Extract from the query.","Need a move-in ready pre-owned property by Oberoi in 'Oberoi Heights'. It's 8 years old, unfurnished, listed at 95 lakhs, tagged as affordable and cozy, oriented west, offers 2 parking facilities, visitor limit of 3 per slot, area of about 1400 sqft, with a mountain view, basic furnishing, building called 'Oberoi Manor', and must have security and power backup.","SELECT * FROM properties WHERE ready_to_movin = TRUE AND developer_name = 'Oberoi' AND project_name = 'Oberoi Heights' AND resale = TRUE AND age = 8 AND furnished ILIKE '%unfurnished%' AND price = 9500000 AND tags ILIKE '%affordable%' AND tags ILIKE '%cozy%' AND facing = 'west' AND parkingCount = 2 AND visitor_perslot = 3 AND area = 1400 AND property_view = 'mountain view' AND property_furnished_type ILIKE '%basic%' AND buildingName = 'Oberoi Manor' AND must_have_tags ILIKE '%security%' AND must_have_tags ILIKE '%power backup%';"
"Extract from the query.","Looking for a brand new, occupancy-ready resale property from Lodha in 'Lodha Riviera'. It’s less than 1 year old, fully furnished, priced at 1.5 crores, described as luxury and elegant, facing north-east, with 3 parking spots, visitor capacity of 5 per slot, carpet area of 2000 sqft, sea view, deluxe furnishing style, building named 'Lodha Grande', and must include a swimming pool and clubhouse.","SELECT * FROM properties WHERE ready_to_movin = TRUE AND developer_name = 'Lodha' AND project_name = 'Lodha Riviera' AND resale = TRUE AND age = 0 AND furnished ILIKE '%fully furnished%' AND price = 15000000 AND tags ILIKE '%luxury%' AND tags ILIKE '%elegant%' AND facing = 'north-east' AND parkingCount = 3 AND visitor_perslot = 5 AND carpet_area = 2000 AND property_view = 'sea view' AND property_furnished_type ILIKE '%deluxe%' AND buildingName = 'Lodha Grande' AND must_have_tags ILIKE '%swimming pool%' AND must_have_tags ILIKE '%clubhouse%';"
"Extract from the query.","Find me a ready-to-move resale property from Hiranandani in 'Hiranandani Gardens'. It is 5 to 7 years old, semi furnished, priced between 70 and 80 lakhs, labeled as premium and well-maintained, facing south, with 2 to 3 parking spots, visitor limit of 2 per slot, area between 1100 and 1200 sqft, garden view, mid-range furnishing style, building called 'Hiranandani Icon', and must have lift and intercom.","SELECT * FROM properties WHERE ready_to_movin = TRUE AND developer_name = 'Hiranandani' AND project_name = 'Hiranandani Gardens' AND resale = TRUE AND age BETWEEN 5 AND 7 AND furnished ILIKE '%semi furnished%' AND price BETWEEN 7000000 AND 8000000 AND tags ILIKE '%premium%' AND tags ILIKE '%well-maintained%' AND facing = 'south' AND parkingCount BETWEEN 2 AND 3 AND visitor_perslot = 2 AND area BETWEEN 1100 AND 1200 AND property_view = 'garden view' AND property_furnished_type ILIKE '%mid-range%' AND buildingName = 'Hiranandani Icon' AND must_have_tags ILIKE '%lift%' AND must_have_tags ILIKE '%intercom%';"
"Extract from the query.","I'm interested in a turnkey resale unit by Prestige in 'Prestige Residency'. It's 6 years old, comes furnished, costs about 85 lakhs, described as upscale with modern interiors, faces northwest, provides 2 parking bays, allows 4 visitors per slot, has an area of 1350 sqft, boasts a skyline view, furnished in executive style, under the building 'Prestige Tower', and must feature a landscaped garden and security surveillance.","SELECT * FROM properties WHERE ready_to_movin = TRUE AND developer_name = 'Prestige' AND project_name = 'Prestige Residency' AND resale = TRUE AND age = 6 AND furnished ILIKE '%furnished%' AND price = 8500000 AND tags ILIKE '%upscale%' AND tags ILIKE '%modern interiors%' AND facing = 'northwest' AND parkingCount = 2 AND visitor_perslot = 4 AND area = 1350 AND property_view = 'skyline view' AND property_furnished_type ILIKE '%executive%' AND buildingName = 'Prestige Tower' AND must_have_tags ILIKE '%landscaped garden%' AND must_have_tags ILIKE '%security surveillance%';"
"Extract from the query.","Search for a newly launched ready-to-move unit by DLF in 'DLF Urban'. It's a fresh launch, 0 years old, comes unfurnished, priced around 1 crore, tagged as contemporary and chic, oriented east-southeast, with one parking spot, visitor limit of 3 per slot, covers an area of 1600 sqft, offers a parkside view, minimalistic furnishing, building named 'DLF Urban Heights', located in Mumbai, and must include eco-friendly features and smart home technology.","SELECT * FROM properties WHERE ready_to_movin = TRUE AND developer_name = 'DLF' AND project_name = 'DLF Urban' AND resale = FALSE AND age = 0 AND furnished ILIKE '%unfurnished%' AND price = 10000000 AND tags ILIKE '%contemporary%' AND tags ILIKE '%chic%' AND facing = 'east-southeast' AND parkingCount = 1 AND visitor_perslot = 3 AND area = 1600 AND property_view = 'parkside view' AND property_furnished_type ILIKE '%minimalistic%' AND buildingName = 'DLF Urban Heights' AND location ILIKE '%Mumbai%' AND must_have_tags ILIKE '%eco-friendly%' AND must_have_tags ILIKE '%smart home technology%';"
"Extract from the query.","I want a modern home in Mumbai.","SELECT * FROM properties WHERE location ILIKE '%Mumbai%' AND tags ILIKE '%modern%';"
"Extract from the query.","Looking for a cozy property, must be in Bangalore.","SELECT * FROM properties WHERE location ILIKE '%Bangalore%' AND tags ILIKE '%cozy%';"
"Extract from the query.","I need a property with at least 4 bedrooms but no more than 2 bathrooms.","SELECT * FROM properties WHERE bedroom_count >= 4 AND bathroom_count <= 2;"
"Extract from the query.","Seeking a spacious apartment: 3-4 bedrooms and only 1 bathroom.","SELECT * FROM properties WHERE bedroom_count BETWEEN 3 AND 4 AND bathroom_count = 1;"
"Extract from the query.","Show me an affordable flat in Pune.","SELECT * FROM properties WHERE location ILIKE '%Pune%' AND property_furnished_type ILIKE '%flat%' AND tags ILIKE '%affordable%';"
"Extract from the query.","Looking for a luxury property with a chic vibe in Delhi.","SELECT * FROM properties WHERE location ILIKE '%Delhi%' AND tags ILIKE '%luxury%' AND tags ILIKE '%chic%';"
"Extract from the query.","I want a property either in Mumbai or Pune with 2 BHK.","SELECT * FROM properties WHERE location ILIKE ANY (ARRAY['%Mumbai%','%Pune%']) AND bedroom_count = 2;"
"Extract from the query.","I don't want a resale property, only new constructions in Bangalore.","SELECT * FROM properties WHERE location ILIKE '%Bangalore%' AND resale = FALSE;"
"Extract from the query.","I need an apartment with a carpet area of 1000 sqft (roughly 93 m²).","SELECT * FROM properties WHERE carpet_area = 1000;"
"Extract from the query.","Looking for a house with a built-up area of 1500 sqft or 139 m².","SELECT * FROM properties WHERE buildup_area = 1500;"
"Extract from the query.","I want a two-bedroom flat in Chennai with a verandah.","SELECT * FROM properties WHERE bedroom_count = 2 AND property_furnished_type ILIKE '%flat%' AND location ILIKE '%Chennai%' AND balcony_count >= 1;"
"Extract from the query.","Looking for a 2 room + hall in Kochi.","SELECT * FROM properties WHERE bedroom_count = 2 AND description ILIKE '%room + hall%' AND location ILIKE '%Kochi%';"
"Extract from the query.","Show me properties built in the last 5 years in Mumbai.","SELECT * FROM properties WHERE location ILIKE '%Mumbai%' AND age <= 5;"
"Extract from the query.","Looking for a new launch property in Delhi, currently under construction.","SELECT * FROM properties WHERE location ILIKE '%Delhi%' AND tags ILIKE '%new launch%' AND tags ILIKE '%under construction%';"



Overwriting dataset.csv


In [25]:
dataset = load_dataset("csv", data_files={"train": "dataset.csv"}, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

In [27]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.729 GB of memory reserved.


In [28]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62 | Num Epochs = 9
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.222300
2,0.249600
3,0.296300
4,0.185700
5,0.179100
6,0.236100
7,0.218000
8,0.388600
9,0.140300
10,0.150300


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

540.6501 seconds used for training.
9.01 minutes used for training.
Peak reserved memory = 5.836 GB.
Peak reserved memory for training = 1.316 GB.
Peak reserved memory % of max memory = 39.571 %.
Peak reserved memory for training % of max memory = 8.923 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [18]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Extract from the query.", # instruction
        "i want a 3bhk furnished appartments", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExtract from the query.\n\n### Input:\ni want a 3bhk furnished appartments\n\n### Response:\nSELECT * FROM properties WHERE house_type = '3bhk' AND furnished = 'furnished';</s>"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [21]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Extract from the query.", # instruction
        "i want a 3 room appartment furnished", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Extract from the query.

### Input:
i want a 3 room appartment furnished

### Response:
SELECT * FROM properties WHERE house_type = '3bhk' AND furnished = 'furnished';</s>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    ),
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nOne of the most famous tall towers in Paris is the Eiffel Tower. It is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. The Eiffel Tower is one']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q5_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
